<a href="https://colab.research.google.com/github/two-phaseflow/JAR513M/blob/main/Week2_GroupAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **JAR513 Week 2 - Group Assignment**

This notebook will help you blah blah blah.

In [1]:
import pandas as pd

Now using pandas we can read in our file and create a dataframe.

In [3]:
df = pd.read_csv("unobtanium_production_data.csv", skiprows=1)

FileNotFoundError: [Errno 2] No such file or directory: 'unobtanium_production_data.csv'

`read_csv` is a function inside of the pandas library. `skiprows` is an argument. We skip the first row because this first row only provides background information and is not part of the dataset. We can start to investigate the dataset.



In [2]:
df.head()

NameError: name 'df' is not defined